## Project - Bank Churn prediction

### 1st case study - Project 1:
 

#### In this project, we aim to predict the churn for a bank, i.e, given a Bank customer, can we build a classifier which can determine whether they will leave or not using Neural networks?
 
Case file: bank.csv 
 
The points distribution for this case is as follows:

1. Read the dataset
2. Drop the columns which are unique for all users like IDs
3. Distinguish the feature and target set
4. Divide the data set into Train and test sets
5. Normalize the train and test data (2.5 points)
6. Initialize & build the model (10 points)
7. Optimize the model (5 points)
9. Predict the results using 0.5 as a threshold (5 points)
10. Print the Accuracy score and confusion matrix (2.5 points)


In [0]:
!pip install tensorflow==2.0

     |████████████████████████████████| 86.3MB 52kB/s 
     |████████████████████████████████| 450kB 27.5MB/s 
     |████████████████████████████████| 3.8MB 28.0MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=20f7348815b81ed52fa581a2fad9b510af0e0202e104df6581f8cc8b250ab888
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0rc0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorboard 2.2.1
    Uninstalling tensorboard-2.2.1:
      Successfully uninstalled tensorboard-2.2.1
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation

In [1]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [2]:
!pip install scikit-learn==0.21.2

     |████████████████████████████████| 6.7MB 2.4MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [0]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix

In [0]:
seed = 66

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#### Read the dataset

In [0]:
ds = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/bank.csv")

In [7]:
ds.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


#### Drop the columns which are unique for all users like IDs

In [8]:
ds['Geography'].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [0]:
#RowNumber #CustomerId and #Surname are unique hence dropping it
ds = ds.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [10]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


#### Distinguish the feature and target set

In [0]:
X = ds.iloc[:,0:10].values # Credit Score through Estimated Salary
y = ds.iloc[:,10].values # Exited

## George's Challenge
How to do this with `df.loc`?

In [0]:
cols_to_use = [col for col in ds.columns if col != 'Exited']
X = ds.loc[:, cols_to_use]
y = ds.loc[:, 'Exited']

In [13]:
# Encoding categorical (string based) data. Country: there are 3 options: France, Spain and Germany
# This will convert those strings into scalar values for analysis
print(X[:8,1], '... will now become: ')

label_X_country_encoder = LabelEncoder()
X[:,1] = label_X_country_encoder.fit_transform(X[:,1])
print(X[:8,1])

['France' 'Spain' 'France' 'France' 'Spain' 'Spain' 'France' 'Germany'] ... will now become: 
[0 2 0 0 2 2 0 1]


In [14]:
# We will do the same thing for gender. this will be binary in this dataset
print(X[:6,2], '... will now become: ')

label_X_gender_encoder = LabelEncoder()
X[:,2] = label_X_gender_encoder.fit_transform(X[:,2])
print(X[:6,2])

['Female' 'Female' 'Female' 'Female' 'Female' 'Male'] ... will now become: 
[0 0 0 0 0 1]


In [15]:
# The Problem here is that we are treating the countries as one variable with ordinal values (0 < 1 < 2). 
# Therefore, one way to get rid of that problem is to split the countries into respective dimensions.
# Gender does not need this as it is binary

# Converting the string features into their own dimensions. Gender doesn't matter here because its binary
#countryhotencoder = OneHotEncoder(categories = [1]) # 1 is the country column
countryhotencoder = ColumnTransformer([("countries", OneHotEncoder(), [1])], remainder="passthrough")
X = countryhotencoder.fit_transform(X)
#X = countryhotencoder.fit_transform(X).toarray()

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


## George's Challenge
What's the problem of using LabelEncoder to create ordinal values?

## George's Bonus
- What is ColumnTransformer? What does it do?
- https://machinelearningmastery.com/columntransformer-for-numerical-and-categorical-data/
- https://jorisvandenbossche.github.io/blog/2018/05/28/scikit-learn-columntransformer/

In [16]:
X.shape

(10000, 12)

In [17]:
X

array([[1.0, 0.0, 0.0, ..., 1, 1, 101348.88],
       [0.0, 0.0, 1.0, ..., 0, 1, 112542.58],
       [1.0, 0.0, 0.0, ..., 1, 0, 113931.57],
       ...,
       [1.0, 0.0, 0.0, ..., 0, 1, 42085.58],
       [0.0, 1.0, 0.0, ..., 1, 0, 92888.52],
       [1.0, 0.0, 0.0, ..., 1, 0, 38190.78]], dtype=object)

In [0]:
# A 0 on two countries means that the country has to be the one variable which wasn't included 
# This will save us from the problem of using too many dimensions
X = X[:,1:] # Got rid of Spain as a dimension.

## George's Tip
Use `drop='first'`

In [19]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit_transform([['a'], ['b'], ['c'], ['a']]).toarray()

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [20]:
enc = OneHotEncoder(drop='first')
enc.fit_transform([['a'], ['b'], ['c'], ['a']]).toarray()

array([[0., 0.],
       [1., 0.],
       [0., 1.],
       [0., 0.]])

#### Divide the data set into Train and test sets

In [0]:
# Splitting the dataset into the Training and Testing set.

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0)

In [23]:
ds.Exited.value_counts(normalize=True)

0    0.7963
1    0.2037
Name: Exited, dtype: float64

## George's Tip
If data is imbalanced, remember to use the `stratify` parameter to preserve the pattern.

#### Normalize the train and test data

In [0]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## George's Tip
Don't confuse yourself!
- standardize
- normalize

#### Initialize & build the model

In [0]:
# Initializing the ANN
classifier = Sequential()

In [0]:
# The amount of nodes (dimensions) in hidden layer should be the average of input and output layers, in this case 6.
# This adds the input layer (by specifying input dimension) AND the first hidden layer (units)
classifier.add(Dense(activation = 'relu', input_dim = 11, units=6, kernel_initializer='uniform'))

## George's Challenge
What does `input_dim` represent here?

In [0]:
#Add 1st hidden layer
classifier.add(Dense(6, activation='sigmoid', kernel_initializer='uniform'))

In [0]:
# Adding the output layer
# Notice that we do not need to specify input dim. 
# we have an output of 1 node, which is the the desired dimensions of our output (stay with the bank or not)
# We use the sigmoid because we want probability outcomes
classifier.add(Dense(1, activation = 'sigmoid', kernel_initializer='uniform')) 

In [0]:
# Create optimizer with default learning rate
# Compile the model
classifier.compile(optimizer='SGD', loss='mse', metrics=['accuracy'])

In [30]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 72        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 7         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


## George's Challenge
What does `None` mean? Hint, we talked about this last time :) 

In [31]:
classifier.fit(
    X_train, 
    y_train,
    validation_data=(X_test,y_test),
    epochs=100,
    batch_size=32
)

Epoch 1/100
250/250 [==============================] - 1s 4ms/step - loss: 0.2057 - accuracy: 0.7960 - val_loss: 0.1816 - val_accuracy: 0.7975
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.1739 - accuracy: 0.7960 - val_loss: 0.1680 - val_accuracy: 0.7975
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.1664 - accuracy: 0.7960 - val_loss: 0.1640 - val_accuracy: 0.7975
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.1640 - accuracy: 0.7960 - val_loss: 0.1626 - val_accuracy: 0.7975
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.1631 - accuracy: 0.7960 - val_loss: 0.1620 - val_accuracy: 0.7975
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.1627 - accuracy: 0.7960 - val_loss: 0.1618 - val_accuracy: 0.7975
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.1626 - accuracy: 0.7960 - val_loss: 0.1616 - val_accuracy: 0.7975

## George's Tip
Indent like a pro!

#### Predict the results using 0.5 as a threshold

In [32]:
y_pred = classifier.predict(X_test)
print(y_pred)

[[0.20427212]
 [0.20502254]
 [0.2041394 ]
 ...
 [0.2043188 ]
 [0.20470184]
 [0.20467544]]


## George's Challenge
Why does the `predict` method output decimals instead of class labels?

In [33]:
# To use the confusion Matrix, we need to convert the probabilities that a customer will leave the bank into the form true or false. 
# So we will use the cutoff value 0.5 to indicate whether they are likely to exit or not.
y_pred = (y_pred > 0.5)
print(y_pred)

[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]


#### Print the Accuracy score and confusion matrix

In [34]:
cm1 = confusion_matrix(y_test, y_pred)
print(cm1)

[[1595    0]
 [ 405    0]]


In [35]:
accuracy_model1 = ((cm1[0][0]+cm1[1][1])*100)/(cm1[0][0]+cm1[1][1]+cm1[0][1]+cm1[1][0])
print (accuracy_model1, '% of testing data was classified correctly')

79.75 % of testing data was classified correctly


## George's Challenge
What's a better way?

#### Optimize the model

Some important parameters to look out for while optimizing neural networks are:

-Type of architecture

-Number of Layers

-Number of Neurons in a layer

-Regularization parameters

-Learning Rate

-Type of optimization / backpropagation technique to use

-Dropout rate

-Weight sharing

##### Number of Layers:
We will keep it similar to the above model so that we can compare the accuracy.
1 hidden layer.

##### Activation:
input layer: relu becasue we are in an input layer. uses the ReLu activation function for  ϕ
output layer: sigmoid becasue we are in an output layer. uses the Sigmoid activation function for  ϕ . This is used instead of the ReLu function becasue it generates probabilities for the outcome. We want the probability that each customer leaves the bank.

##### Type of optimization / backpropagation technique to use: 
We will use Adam. Adam is a very efficeint variation of Stochastic Gradient Descent. For Adam and its variant, learning rate or the decay rate does not really matter too much.

##### Learning Rate:
default learning rate 0.001.

##### Number of Neurons in a layer:
We will keep it 6 as per our initial calculation above.

##### Weight sharing / kernel_initializer: 
uniform the distribution with which we randomly initialize weights for the nodes in this layer.

##### Loss:
loss: binary_crossentropy This is the loss function used within adam. This should be the logarthmic loss. If our dependent (output variable) is Binary, it is binary_crossentropy. If Categorical, then it is called categorical_crossentropy

Rebuilding the model using these optimised parameters

## George's Tip
https://stats.stackexchange.com/questions/260505/should-i-use-a-categorical-cross-entropy-or-binary-cross-entropy-loss-for-binary

In [0]:
# Initializing the ANN
Oclassifier = Sequential()
# This adds the input layer (by specifying input dimension) AND the first hidden layer (units)
Oclassifier.add(Dense(activation = 'relu', input_dim = 11, units=6, kernel_initializer='uniform'))

In [0]:
# Adding the hidden layer
# Notice that we do not need to specify input dim. 
Oclassifier.add(Dense(activation = 'relu', units=6, kernel_initializer='uniform')) 

## George's Challenge
Why don't we have to specify `input dim`?

In [0]:
# Adding the output layer
# Notice that we do not need to specify input dim. 
# we have an output of 1 node, which is the the desired dimensions of our output (stay with the bank or not)
# We use the sigmoid because we want probability outcomes
Oclassifier.add(Dense(activation = 'sigmoid', units=1, kernel_initializer='uniform')) 

In [0]:
Oclassifier.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [41]:
Oclassifier.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 6)                 72        
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 7         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [42]:
Oclassifier.fit(X_train, y_train,           
          validation_data=(X_test,y_test),
          epochs=100,
          batch_size=32)

Epoch 1/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5688 - accuracy: 0.7958 - val_loss: 0.4456 - val_accuracy: 0.7975
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4377 - accuracy: 0.7960 - val_loss: 0.4320 - val_accuracy: 0.7975
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4307 - accuracy: 0.7960 - val_loss: 0.4287 - val_accuracy: 0.7975
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4276 - accuracy: 0.7960 - val_loss: 0.4261 - val_accuracy: 0.7975
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4248 - accuracy: 0.7960 - val_loss: 0.4225 - val_accuracy: 0.7975
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4217 - accuracy: 0.7960 - val_loss: 0.4190 - val_accuracy: 0.7975
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4200 - accuracy: 0.8111 - val_loss: 0.4168 - val_accuracy: 0.8185

#### Predict the results using 0.5 as a threshold

In [43]:
y_pred = Oclassifier.predict(X_test)
print(y_pred)

[[0.2265449 ]
 [0.37242636]
 [0.18577732]
 ...
 [0.20016901]
 [0.12504056]
 [0.14754216]]


In [44]:
# To use the confusion Matrix, we need to convert the probabilities that a customer will leave the bank into the form true or false. 
# So we will use the cutoff value 0.5 to indicate whether they are likely to exit or not.
y_pred = (y_pred > 0.5)
print(y_pred)

[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]


#### Print the Accuracy score and confusion matrix

In [45]:
cm2 = confusion_matrix(y_test, y_pred)
print(cm2)

[[1541   54]
 [ 258  147]]


In [46]:
accuracy_model2 = ((cm2[0][0]+cm2[1][1])*100)/(cm2[0][0]+cm2[1][1]+cm2[0][1]+cm2[1][0])
print (accuracy_model2, '% of testing data was classified correctly')

84.4 % of testing data was classified correctly


#### The optimised model gives better results as seen from the confusion matrix as well as improved accuracy.


We encourage the learners to further look for optimizing the model and come up with better results.

#### Happy Learning!

# George's Bonus $$$
- DL and ML's use cases
- DL model tuning with sklearn

In [42]:
# Tune the DL model
import numpy as np
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

def create_model(
    input_units=6, 
    hidden_units=6, 
    optimizer='adam', 
    init='uniform'
):
  model = Sequential()
  model.add(Dense(
      activation='relu', 
      input_dim=11, 
      units=input_units, 
      kernel_initializer=init
      ))
  model.add(Dense(
      activation='relu', 
      units=hidden_units, 
      kernel_initializer=init
      )) 
  model.add(Dense(
      activation='sigmoid', 
      units=1, 
      kernel_initializer=init
      )) 
  model.compile(
      optimizer=optimizer, 
      loss='binary_crossentropy', 
      metrics=['accuracy']
      )
  return model

model = KerasClassifier(build_fn=create_model, verbose=0)

input_units = np.arange(2, 30)
hidden_units = np.arange(2, 30)
optimizer = ['rmsprop', 'adam']
init = ['glorot_uniform', 'normal', 'uniform']
epochs = np.arange(10, 150, 10)
batch_size = [32, 64, 128, 256]

param_grid = dict(
    input_units=input_units,
    hidden_units=hidden_units, 
    optimizer=optimizer, 
    init=init,
    epochs=epochs,
    batch_size=batch_size
)
gs = RandomizedSearchCV(
    estimator=model, 
    param_distributions=param_grid, 
    n_iter=60,
    random_state=seed,
    verbose=2
  )
gs.fit(X_train, y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
[CV] optimizer=adam, input_units=5, init=uniform, hidden_units=22, epochs=20, batch_size=128 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  optimizer=adam, input_units=5, init=uniform, hidden_units=22, epochs=20, batch_size=128, total=   1.7s
[CV] optimizer=adam, input_units=5, init=uniform, hidden_units=22, epochs=20, batch_size=128 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.7s remaining:    0.0s


[CV]  optimizer=adam, input_units=5, init=uniform, hidden_units=22, epochs=20, batch_size=128, total=   1.7s
[CV] optimizer=adam, input_units=5, init=uniform, hidden_units=22, epochs=20, batch_size=128 
[CV]  optimizer=adam, input_units=5, init=uniform, hidden_units=22, epochs=20, batch_size=128, total=   1.6s
[CV] optimizer=adam, input_units=5, init=uniform, hidden_units=22, epochs=20, batch_size=128 
[CV]  optimizer=adam, input_units=5, init=uniform, hidden_units=22, epochs=20, batch_size=128, total=   1.7s
[CV] optimizer=adam, input_units=5, init=uniform, hidden_units=22, epochs=20, batch_size=128 
[CV]  optimizer=adam, input_units=5, init=uniform, hidden_units=22, epochs=20, batch_size=128, total=   1.6s
[CV] optimizer=adam, input_units=23, init=glorot_uniform, hidden_units=19, epochs=20, batch_size=128 
[CV]  optimizer=adam, input_units=23, init=glorot_uniform, hidden_units=19, epochs=20, batch_size=128, total=   2.0s
[CV] optimizer=adam, input_units=23, init=glorot_uniform, hidde

[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed: 55.2min finished


RuntimeError: ignored

In [43]:
print(f'Best score: {gs.best_score_}')
print(f'Best params: {gs.best_params_}')

Best score: 0.8605000019073487
Best params: {'optimizer': 'adam', 'input_units': 17, 'init': 'uniform', 'hidden_units': 11, 'epochs': 140, 'batch_size': 64}


In [47]:
gs.best_estimator_

AttributeError: ignored

In [47]:
# Spot checking with ML algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

lr = LogisticRegression(random_state=seed)
dt = DecisionTreeClassifier(random_state=seed)
rf = RandomForestClassifier(random_state=seed)
svc = SVC(random_state=seed)

models = dict(lr=lr, dt=dt, rf=rf, svc=svc)
#results = []

for model_name, model in models.items():
  model.fit(X_train, y_train)
  result = model.score(X_test, y_test)
  print(f'{model_name}: {result:.2f}')

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


lr: 0.81
dt: 0.80
rf: 0.86
svc: 0.86
